In [2]:
!pip install ultralytics streamlit opencv-python-headless pandas cloudflared

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.2 MB/s eta 0:00:00
  Created wheel for cloudflared: filename=cloudflared-1.0.0.2-py3-none-any.whl size=2983 sha256=d21dee35d9ca9163a3bb53eacf4e7096e29340a6a67734085886e8fac7014e6c
  Stored in directory: /root/.cache/pip/wheels/5b/ec/09/c3bcd3470be046ec77a9c0cb9d8bb6ceed49c831460878ab0a
Successfully built cloudflared


In [6]:
!pip install cloudflared


In [7]:
!apt install cloudflared -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cloudflared


In [8]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


Selecting previously unselected package cloudflared.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.9.1) ...
Setting up cloudflared (2025.9.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [9]:
!cloudflared --version


cloudflared version 2025.9.1 (built 2025-09-22-13:28 UTC)


In [3]:
from google.colab import files
import os

os.makedirs("models", exist_ok=True)

uploaded = files.upload()
for fn in uploaded.keys():
    os.rename(fn, f"models/{fn}")
print("✅ Model uploaded to /models folder")

Saving best.pt to best.pt
✅ Model uploaded to /models folder


In [4]:
%%writefile app.py
import streamlit as st
import cv2, tempfile, os, pandas as pd, time
from ultralytics import YOLO
from moviepy.editor import VideoFileClip, ImageSequenceClip
import numpy as np

st.set_page_config(page_title="Vehicle Detection - YOLOv8", layout="wide")
st.title("Vehicle Detection using YOLOv8")
st.markdown("Upload YOLOv8 model and video to detect vehicles, count classes, and view accuracy.")

# Upload model
uploaded_model = st.file_uploader("Upload YOLOv8 model (.pt)", type=["pt"])
model = None
if uploaded_model:
    os.makedirs("models", exist_ok=True)
    model_path = os.path.join("models", uploaded_model.name)
    with open(model_path, "wb") as f:
        f.write(uploaded_model.read())
    st.success(f"Model {uploaded_model.name} uploaded successfully!")
    model = YOLO(model_path)

# Upload video
uploaded_video = st.file_uploader("Upload video", type=["mp4", "avi", "mov"])
if uploaded_video and model:
    temp_video = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    temp_video.write(uploaded_video.read())
    temp_video.flush()
    st.video(temp_video.name)

    if st.button("Run Detection"):
        st.info("Running detection... ⏳")
        progress_bar = st.progress(0)
        status_text = st.empty()

        cap = cv2.VideoCapture(temp_video.name)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frames = []

        vehicle_counts = {}
        confidences = []

        for frame_no in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, verbose=False)
            annotated = results[0].plot()
            frames.append(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))

            boxes = results[0].boxes
            if boxes is not None and len(boxes) > 0:
                for cls_id, conf in zip(boxes.cls.cpu().numpy(), boxes.conf.cpu().numpy()):
                    label = model.names[int(cls_id)]
                    vehicle_counts[label] = vehicle_counts.get(label, 0) + 1
                    confidences.append(float(conf))

            progress = int((frame_no + 1) / total_frames * 100)
            progress_bar.progress(progress)
            status_text.text(f"Processing frame {frame_no + 1}/{total_frames}")

        cap.release()
        cv2.destroyAllWindows()
        time.sleep(1)

        # Use MoviePy to save properly encoded video
        temp_output = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
        clip = ImageSequenceClip(frames, fps=fps)
        clip.write_videofile(temp_output.name, codec='libx264', audio=False, verbose=False, logger=None)

        st.success("✅ Detection complete!")
        st.video(temp_output.name)

        if vehicle_counts:
            st.subheader("Vehicle Count by Class")
            st.table(pd.DataFrame(vehicle_counts.items(), columns=["Vehicle Type", "Count"]))

        if confidences:
            avg_conf = sum(confidences) / len(confidences)
            st.subheader("Detection Accuracy")
            st.metric(label="Average Confidence", value=f"{avg_conf*100:.2f}%")

        with open(temp_output.name, "rb") as f:
            st.download_button("Download Processed Video", f, file_name="detected_output.mp4")


Writing app.py


In [ ]:
# Kill previous instances
!pkill -f streamlit || true
!pkill -f cloudflared || true

# Start Streamlit in background
!nohup streamlit run app.py --server.port 8501 &>/content/log.txt &

# Wait for Streamlit to boot
import time
time.sleep(5)

# Start Cloudflared tunnel
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate

^C
^C
2025-10-10T16:58:10Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-10T16:58:10Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-10T16:58:14Z INF +--------------------------------------------------------------------------------------------+
2025-10-10T16:58:14Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-10T16:58:14Z INF |  https://dsl-term-beyond-coordinate.trycloudflar